# **Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups**

## -  **Consignas:**

## **1**. **Vectorizar documentos**.
 Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.


## **2**. **Entrenar modelos de clasificación Naïve Bayes**
para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámetros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

## **3**. **Transponer la matriz documento-término**.
De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [ ]:
# Importo las librerías necesarias
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# Se utiliza la función "fetch_20newsgroups" para cargar el subconjunto de entrenamiento (train) del dataset "20 newsgroups",
# removiendo encabezados, pies de página y citas de los correos electrónicos
newsgroups_train = fetch_20newsgroups(subset="train", remove=("headers", "footers", "quotes"))

# Instanciar el vectorizador TF-IDF
# Este vectorizador transformará los textos en una matriz esparsa en la que cada fila representará un documento, y cada columna
# representará un término (palabra o token) con su valor de TF-IDF.
tfidfvect = TfidfVectorizer()

# Accesso al texto
newsgroups_train.data[0]



'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# Fit-transformar los datos de entrenamiento, ajusta el vectorizador (fit) para aprender el vocabulario de los documentos y luego transforma los textos en una matriz TF-IDF
X_train = tfidfvect.fit_transform(newsgroups_train.data)

# Obtener el vocabulario
vocabulary = tfidfvect.vocabulary_

# Mostrar algunas palabras del vocabulario
words = list(vocabulary.keys()) # Convierte el conjunto de claves del diccionario vocabulary en una lista de palabras
print(f"Cantidad total de palabras en el vocabulario: {len(words)}") # Imprime el número total de palabras en el vocabulario
print("Primeras palabras en el vocabulario:", words[:20])  # Muestra las primeras 20 palabras
print(f"shape: {X_train.shape}")
print(f"cantidad de documentos: {X_train.shape[0]}")
print(f"cantidad de términos: {X_train.shape[1]}")

Cantidad total de palabras en el vocabulario: 101631
Primeras palabras en el vocabulario: ['was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'door', 'sports', 'looked']
shape: (11314, 101631)
cantidad de documentos: 11314
cantidad de términos: 101631


In [ ]:

# Se guarda el vector target que contiene las etiquetas de clasificación (categorías) de cada documento en el conjunto de entrenamiento.
y_train = newsgroups_train.target
y_train[:15]




array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4,  8, 19,  4, 14,  6])

In [ ]:
# Muestro las diferentes clases de cada grupo de noticias
print(f"clases {np.unique(newsgroups_train.target)}")
print("="*68)
newsgroups_train.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
# Seleccionar aleatoriamente 5 documentos del conjunto de entrenamiento (sin reemplazo)
np.random.seed(6)  # Semilla para repetir la elección aleatoria
random_docs = np.random.choice(X_train.shape[0], 5, replace=False)

# Iterar sobre cada índice seleccionado aleatoriamente en random_docs e imprimir la información
for idx in random_docs:
    print(f"\n\033[1mDocumento original ID: {idx}\033[0m\n")  # Texto en negrita para resaltar el documento

    # Mostrar la categoría del documento original (Color azul)
    print(f"Categoría del documento original: \033[94m{newsgroups_train.target_names[y_train[idx]]}\033[0m\n")

    # Mostrar una parte del contenido del documento original (solo los primeros 120 caracteres)
    print(f"Contenido (primeros 100 caracteres): {newsgroups_train.data[idx][:100]}...\n")

    # Calcular la similaridad coseno entre el documento seleccionado y todos los demás
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Obtener los 5 documentos más similares (excluyendo el propio documento)
    mostsim = np.argsort(cossim)[::-1][1:6]

    # Mostrar los documentos más similares, sus categorías y valores de similaridad
    print(f"Documentos más similares:\n")
    for i, sim_idx in enumerate(mostsim, start=1):
        similarity_value = cossim[sim_idx]  # Valor de similaridad coseno
        print(f"\033[1mDocumento ID {sim_idx}:\033[0m")  # Negrita para el título
        print(f"Categoría: \033[92m{newsgroups_train.target_names[y_train[sim_idx]]}\033[0m")  # Categoría en color verde
        print(f"Valor de similaridad: \033[93m{similarity_value:.4f}\033[0m")  # Valor de similaridad en color amarillo
        print(f"Contenido del documento (primeros 120 caracteres): {newsgroups_train.data[sim_idx][:120]}...\n")

    # Separador visual para cada iteración
    print("<>" * 70)



Documento original ID: 2097

Categoría del documento original: sci.med

Contenido (primeros 100 caracteres): 
His _heart_? This jerk doesn't have a heart, and it beats me why you're
apologizing for him. In my ...

Documentos más similares:

Documento ID 8135:
Categoría: sci.med
Valor de similaridad: 0.2430
Contenido del documento (primeros 120 caracteres): I need advice with a situation which occurred between me and a physican
which upset me.  I saw this doctor for a problem...

Documento ID 9670:
Categoría: soc.religion.christian
Valor de similaridad: 0.2313
Contenido del documento (primeros 120 caracteres): 
[..]


Hello.  Firstly, what do you exactly mean by "fundamentalist"?  I will
for the time being assume that what you m...

Documento ID 5829:
Categoría: talk.politics.mideast
Valor de similaridad: 0.2299
Contenido del documento (primeros 120 caracteres): Accounts of Anti-Armenian Human Right Violations in Azerbaijan #013
                 Prelude to Current Events in Nagorn...



# 2. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test.

 Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [ ]:
# Importar las librerías necesarias
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Cargar el dataset "20 newsgroups" (subset train)
newsgroups_train = fetch_20newsgroups(subset="train", remove=("headers", "footers", "quotes"))

# Instanciar el vectorizador TF-IDF
tfidfvect = TfidfVectorizer()

# Fit-transformar los datos de entrenamiento
X_train = tfidfvect.fit_transform(newsgroups_train.data)

# Guardar las etiquetas de clasificación
y_train = newsgroups_train.target

# Separar el conjunto de entrenamiento en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Entrenar y evaluar modelos Naïve Bayes
models = {
    "MultinomialNB": MultinomialNB(),
    "ComplementNB": ComplementNB()
}

for model_name, model in models.items():
    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Realizar predicciones sobre el conjunto de validación
    y_pred = model.predict(X_val)

    # Calcular el F1-score macro
    f1 = f1_score(y_val, y_pred, average="macro")

    # Imprimir el F1-score
    print(f"{model_name} F1 Score: {f1:.4f}")


MultinomialNB F1 Score: 0.6200
ComplementNB F1 Score: 0.7493





# **3**. Transponer la matriz documento-término.
De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [ ]:

# Transponer la matriz documento-término para obtener una matriz término-documento
X_train_transposed = X_train.T

# Obtener el vocabulario
vocabulary = tfidfvect.vocabulary_

# Mostrar algunas palabras del vocabulario
words = list(vocabulary.keys()) # Convierte el conjunto de claves del diccionario vocabulary en una lista de palabras
print(f"Cantidad total de palabras en el vocabulario: {len(words)}") # Imprime el número total de palabras en el vocabulario
print("Primeras palabras en el vocabulario:", words[:100],"\n")  # Muestra las primeras 20 palabras

# Elegir manualmente 5 palabras para evaluar su similaridad
palabras = ["name", "door", "made", "mail", "number"]

# Crear un diccionario inverso para mapear índices a palabras
idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}

# Calcular y mostrar las palabras más similares para cada palabra elegida
for palabra in palabras:
    if palabra in tfidfvect.vocabulary_:
        # Obtener el índice de la palabra en la matriz transpuesta
        idx = tfidfvect.vocabulary_[palabra]

        # Obtener el vector de la palabra seleccionada
        palabra_vect = X_train_transposed[idx]

        # Calcular la similaridad coseno entre la palabra seleccionada y todas las demás
        similar_words = cosine_similarity(palabra_vect, X_train_transposed)[0]

        # Obtener los 5 índices de palabras más similares (excluyendo la palabra misma)
        most_similar_idx = np.argsort(similar_words)[::-1][1:6]

        # Mostrar los resultados
        print(f"\033[1mPalabra elegida: {palabra}\033[0m")
        for idx in most_similar_idx:
            print(f"Palabra similar encontrada: {idx2word[idx]}")
        print("-" * 70)
    else:
        print(f"Palabra "{palabra}" no encontrada en el vocabulario.")


Cantidad total de palabras en el vocabulario: 101631
Primeras palabras en el vocabulario: ['was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'door', 'sports', 'looked', 'to', 'be', 'from', 'late', '60s', 'early', '70s', 'called', 'bricklin', 'doors', 'were', 'really', 'small', 'in', 'addition', 'front', 'bumper', 'separate', 'rest', 'of', 'body', 'is', 'all', 'know', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'where', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'funky', 'looking', 'please', 'mail', 'fair', 'number', 'brave', 'souls', 'who', 'upgraded', 'their', 'si', 'clock', 'oscillator', 'shared', 'experiences', 'for', 'poll', 'send', 'brief', 'message', 'detailing', 'your', 'with', 'procedure', 'top', 'speed', 'attained', 'cpu', 'rated', 'add', 'cards', 'and', 'adapters', 'heat', 'sinks', 'hour', 'usage', 'per', 'floppy'] 

Palabra elegida: name
Pal




# **4**. Prueba ajustando parámetros de instanciación del vectorizador y modelos
Para explorar cómo los parámetros de instanciación del vectorizador y los modelos afectan el desempeño del modelo. Aquí lo que se probó fue:
1. Modificar parámetros del TfidfVectorizer, ya que tiene varios parámetros que pueden influir en el desempeño del modelo, tales como:
- max_features: limitar el número máximo de términos (palabras).
- ngram_range: cambiar el rango de n-gramas (por ejemplo, usar bigramas).
- max_df y min_df: para eliminar palabras demasiado comunes o demasiado raras.
2. Modificar parámetros de los modelos Naïve Bayes, tanto MultinomialNB como ComplementNB tienen el parámetro:

- alpha: para suavizar las probabilidades y evitar que los valores sean cero (suavizado de Laplace).

In [ ]:
# Importar las librerías necesarias
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Cargar el dataset "20 newsgroups" (subset train)
newsgroups_train = fetch_20newsgroups(subset="train", remove=("headers", "footers", "quotes"))

# Configuración del vectorizador 1: Unigrams con 5000 términos
tfidfvect1 = TfidfVectorizer(max_features=5000, ngram_range=(1, 1))

# Configuración del vectorizador 2: Unigrams + Bigrams con 5000 términos
tfidfvect2 = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

# Configuración del vectorizador 3: Solo unigrams con 10000 términos
tfidfvect3 = TfidfVectorizer(max_features=10000, ngram_range=(1, 1))

# Lista de vectorizadores a probar
vectorizers = [tfidfvect1, tfidfvect2, tfidfvect3]

# Modelos Naïve Bayes
multinomial_nb1 = MultinomialNB(alpha=1.0)  # Multinomial con alpha=1
multinomial_nb2 = MultinomialNB(alpha=0.5)  # Multinomial con alpha=0.5
complement_nb1 = ComplementNB(alpha=1.0)    # Complement con alpha=1
complement_nb2 = ComplementNB(alpha=0.5)    # Complement con alpha=0.5

# Lista de modelos a probar
models = [multinomial_nb1, multinomial_nb2, complement_nb1, complement_nb2]

# Entrenar y evaluar combinaciones de vectorizadores y modelos
for i, vectorizer in enumerate(vectorizers, start=1):
    print(f"\nProbando vectorizador {i}...\n")

    # Fit-transformar los datos de entrenamiento
    X_train = vectorizer.fit_transform(newsgroups_train.data)

    # Guardar las etiquetas de clasificación
    y_train = newsgroups_train.target

    # Separar el conjunto de entrenamiento en conjuntos de entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Entrenar y evaluar cada modelo Naïve Bayes
    for x, model in enumerate(models, start=1):
        # Entrenar el modelo
        model.fit(X_train, y_train)

        # Realizar predicciones sobre el conjunto de validación
        y_pred = model.predict(X_val)

        # Calcular el F1-score macro
        f1 = f1_score(y_val, y_pred, average="macro")

        # Imprimir el F1-score
        model_name = f"MultinomialNB {x}" if x <= 2 else f"ComplementNB {x-2}"
        print(f"{model_name} con Vectorizador {i} F1 Score: {f1:.4f}")



Probando vectorizador 1...

MultinomialNB 1 con Vectorizador 1 F1 Score: 0.6448
MultinomialNB 2 con Vectorizador 1 F1 Score: 0.6717
ComplementNB 1 con Vectorizador 1 F1 Score: 0.6827
ComplementNB 2 con Vectorizador 1 F1 Score: 0.6827

Probando vectorizador 2...

MultinomialNB 1 con Vectorizador 2 F1 Score: 0.6150
MultinomialNB 2 con Vectorizador 2 F1 Score: 0.6295
ComplementNB 1 con Vectorizador 2 F1 Score: 0.6380
ComplementNB 2 con Vectorizador 2 F1 Score: 0.6362

Probando vectorizador 3...

MultinomialNB 1 con Vectorizador 3 F1 Score: 0.6518
MultinomialNB 2 con Vectorizador 3 F1 Score: 0.6891
ComplementNB 1 con Vectorizador 3 F1 Score: 0.7180
ComplementNB 2 con Vectorizador 3 F1 Score: 0.7144


## Proyecto de análisis de texto
Se utiliza como base el conjunto de datos "20 Newsgroups", que contiene documentos de texto distribuidos en diversas categorías temáticas . El objetivo principal fue procesar este conjunto de datos utilizando técnicas de vectorización de texto, de cálculo de similitud entre documentos y de entrenamiento de modelos de clasificación para analizar luego su rendimiento.


Para convertir los textos en representaciones numéricas, se utilizó el vectorizador TF-IDF (Term Frequency-Inverse Document Frequency). Este método transforma los textos en una matriz esparsa, donde cada fila corresponde a un documento y cada columna a un término del vocabulario. En este caso, se generó un vocabulario de 101,631 palabras, lo que indica un amplio rango de términos presentes en el conjunto de datos.

Para lograr una mejor comprensión y eficiencia en la lectura de datos, se eliminaron elementos no relevantes como encabezados, pies de página y citas, lo que permite concentrarse exclusivamente en el contenido relevante de cada documento.


Hecho esto, se analiza la similitud entre documentos, seleccionando al azar 5 documentos del conjunto de datos y calculando su "similaridad coseno" con el resto. Este cálculo permite medir cuán parecidos son dos documentos en cuanto a los términos que comparten.

A partir de los resultados, se identificaron los 5 documentos más similares para cada uno de los seleccionados. Se imprimieron las categorías de los documentos, su índice, la similaridad y un fragmento de su contenido, lo que proporciona una visión general del texto.

## Entrenamiento de modelos de clasificación
Una parte clave del análisis fue entrenar modelos de clasificación para predecir las categorías de los documentos. Se trabajó con dos algoritmos de Naive Bayes:

- MultinomialNB
- ComplementNB

Estos modelos se entrenaron utilizando el conjunto de datos transformado mediante TF-IDF, y su rendimiento se evaluó utilizando el F1-score macro. Este tipo de métrica permite evaluar el rendimiento global de los modelos teniendo en cuenta tanto la precisión como la exhaustividad para cada clase.

### Resultados de los modelos
MultinomialNB obtuvo un F1-score de 0.5854, es decir un rendimiento medio.
ComplementNB logró un F1-score de 0.6930, mejorando el desempeño en comparación con el modelo anterior.
El mejor rendimiento del modelo ComplementNB sugiere que es más adecuado para manejar clases desbalanceadas en este conjunto de datos, una situación común en problemas de clasificación de texto.

### Análisis de similaridad de términos
Además de analizar documentos, se llevó a cabo un estudio de similaridad de palabras. Se transpuso la matriz documento-término para obtener una matriz término-documento, lo que permitió calcular la similaridad coseno entre las palabras del vocabulario.

Se eligieron 5 palabras ("name", "door", "made", "mail", "number") para evaluar su similaridad con otras palabras. Para cada una de ellas, se identificaron las 5 palabras más similares, lo que permite explorar relaciones entre términos dentro del corpus de documentos.

## Ajuste de parámetros de instanciación
El ajuste de los parámetros tanto en los vectorizadores como en los modelos Naive Bayes influye de manera significativa en el rendimiento del sistema de clasificación.
En la configuración del Vectorizador TF-IDF los parámetros ajustados en los diferentes vectorizadores fueron:
- Número máximo de términos (max_features) con el que podemos limitar o ampliar el tamaño de las representaciones de texto.
- Rango de N-gramas (ngram_range)que define si el vectorizador considera solo palabras individuales (unigrams) o también combinaciones de palabras (bigrams).Los unigrams tienden a capturar información semántica básica, mientras que los bigrams pueden captar patrones o relaciones entre palabras.

En la configuración de los Modelos Naive Bayes
el parámetro clave ajustado fue:

- alpha: El parámetro de suavización de Laplace. Este parámetro controla cómo se manejan las características (palabras) que no están presentes en una clase determinada. Un valor de alpha=1 implica mayor suavización, mientras que valores menores (como alpha=0.5) permiten que las probabilidades estén más influenciadas por las frecuencias observadas.

## **Conclusiones personales**
Este proyecto en mi caso me obligó a la búsqueda en línea de material (mayormente videos) que me terminen de aclarar el tema, ya que no tenía ninguna experiencia previa. He visto que las técnicas de Procesamiento de Lenguaje Natural que comenzamos a utilizar necesitan de mucha prueba y ajuste para lograr un resultado eficiente y depende en gran medida de la estructura del texto a analizar.

Viendo como ComplementNB supera a MultinomialNB en este conjunto de datos donde las clases no están balanceadas, se nota la importancia de seleccionar el algoritmo adecuado para mejorar el rendimiento en la clasificación de textos.

El ajuste de los parámetros del vectorizador y de los modelos de Naive Bayes impacta  en el rendimiento del sistema de clasificación. En este caso el ajuste del parámetro Alpha en la instanciación de modelos no afectó prácticamente al rendimiento, si lo hizo el ajuste del vectorizador, donde utilizar sólo unigrams con un mayor número de términos ayudó a capturar más información contextual del texto, mejorando la precisión.









